In [2]:
from keras.models import Model
from keras.layers import Lambda,Activation,Dense,Conv2D,Input,BatchNormalization,MaxPooling2D,Flatten
import keras.backend as K
import numpy as np

In [21]:
input_tensor_1 = Input([32,32,3])
input_tensor_2 = Input([4,])
input_target = Input([2,])

In [22]:
#loss 计算函数

#mean 求平均 sum求和 square 平方 abs 绝对值
def cus_loss1(y_true,y_pred):
    return K.mean(K.abs(y_true-y_pred))
    
def cus_loss2(y_true,y_pred):
    return K.mean(K.abs(y_true-y_pred))

In [28]:
#网络结构
#第一组
x = BatchNormalization(axis=1,name='BN-1')(input_tensor_1)

x = Conv2D(filters = 32,kernel_size = (3,3),padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x = Conv2D(filters = 32,kernel_size = (3,3),padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)

x = Flatten()(x)
x = Dense(units=16)(x)
out2 = Dense(units=2)(x)

#第二组
y = Dense(units = 32) (input_tensor_2)
out1 = Dense(units =2 )(y)

#第三组
z = Dense(units = 8)(input_target)
out3 = Dense(units=2)(z)

#自定义loss
loss1 = Lambda(lambda x:cus_loss1(*x),name='loss1')([out2,out1]) # Realdata - pred data = loss
loss2 = Lambda(lambda x:cus_loss2(*x),name='loss2')([out3,out2])


In [29]:
model = Model(input=[input_tensor_1,input_tensor_2,input_target],output=[out1,out2,out3,loss1,loss2])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [30]:
#获取loss
loss_layer1 = model.get_layer('loss1').output
loss_layer2 = model.get_layer('loss2').output

#向模型添加loss
model.add_loss(loss_layer1)
model.add_loss(loss_layer2)

In [31]:
#loss +None 表述不需要梯度返回， 有多少个 out 就有多少个none
model.compile(optimizer='sgd',loss=[None,None,None,None,None])

In [38]:
#dataset
def data_gen(number):
    for i in range(number):
        yield [np.random.normal(1,1,size=(1,32,32,3)),
                            np.random.normal(1,1,size=(1,4)),
                                             np.random.normal(1,1,size=(1,2))],[]
dataset = data_gen(1000)

In [39]:
train = model.fit_generator(dataset,epochs=10,steps_per_epoch=20)

Epoch 1/10
20/20 [==============================] - 7s 353ms/step - loss: 2.1724
Epoch 2/10
20/20 [==============================] - 0s 12ms/step - loss: 0.7822
Epoch 3/10
20/20 [==============================] - 0s 12ms/step - loss: 0.4515
Epoch 4/10
20/20 [==============================] - 0s 13ms/step - loss: 0.2656
Epoch 5/10
20/20 [==============================] - 0s 13ms/step - loss: 0.1545
Epoch 6/10
20/20 [==============================] - 0s 12ms/step - loss: 0.1432
Epoch 7/10
20/20 [==============================] - 0s 12ms/step - loss: 0.1468
Epoch 8/10
20/20 [==============================] - 0s 12ms/step - loss: 0.1247
Epoch 9/10
20/20 [==============================] - 0s 12ms/step - loss: 0.1316
Epoch 10/10
20/20 [==============================] - 0s 12ms/step - loss: 0.1121


In [40]:
from keras.utils.vis_utils import plot_model
plot_model(model=model,to_file='model_plain-loss.png',show_shapes=True)